In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import gym
import torch
import random

from collections import namedtuple
from itertools import count
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T
from torch.utils.data import TensorDataset
import copy
import gym_minigrid
from soccer_env import soccer
import cvxpy as cp

In [7]:
#chicken game
r_a = np.array([[6,2],[7,0]])
r_b = np.array([[6,7],[2,0]])
r_b = r_b.T
n = r_a.shape[0]
constraints = []
#print(r)
# row player is A column player is B
pa = cp.Variable((n,n)) 
constraints = []
#constraints regarding equilibrium
for a in range(n):
    q_diff = np.empty([n,0])
    for i in range(n):
        q_diff = np.append(q_diff,np.array([r_a[a,:]-r_a[i,:]]).T,axis = 1)
        #print(q_diff)
    lhs = pa[a,:]@q_diff
    print(lhs)
    constraints = constraints + [lhs>=np.zeros(n)]

pb = pa.T
for b in range(n):
    q_diff = np.empty([n,0])
    for i in range(n):
        q_diff = np.append(q_diff,np.array([r_b[b,:]-r_b[i,:]]).T,axis = 1)
        print(q_diff)
    lhs = pb[b,:]@q_diff
    print(lhs)
    constraints = constraints + [lhs>=np.zeros(n)]
#constraints around pa itself
#np.full([5,5],0.000000000000001)
constraints = constraints + [pa>= 0]
constraints = constraints + [cp.sum(pa) == 1]
#constraints: CEQ has 5*4*2 + 25 + 1 constraints and one objective. maximize(p*(q1 + q2))
# utilitarian ce q objective: maximize combined expected values
objective = cp.Maximize(cp.sum(cp.multiply(pa,r_a)+cp.multiply(pb,r_b)))
problem = cp.Problem(objective,constraints)
#solver='ECOS_BB'
result = problem.solve()
p = pa.value
print('p',p)
v_a =  np.multiply(p,r_a)
v_b = np.multiply(p.T,r_b)  
print(result)

var194[0, 0:2] @ [[ 0. -1.]
 [ 0.  2.]]
var194[1, 0:2] @ [[ 1.  0.]
 [-2.  0.]]
[[0.]
 [0.]]
[[ 0. -1.]
 [ 0.  2.]]
var194.T[0, 0:2] @ [[ 0. -1.]
 [ 0.  2.]]
[[ 1.]
 [-2.]]
[[ 1.  0.]
 [-2.  0.]]
var194.T[1, 0:2] @ [[ 1.  0.]
 [-2.  0.]]
p [[ 5.00000000e-01  2.50000000e-01]
 [ 2.50000000e-01 -2.30952415e-10]]
10.500000003126665
